# Интро и полезная литература

Данный файл является скорее демонстрацией того, в чем суть Word2Vec и что он умеет делать. Этот файл - песочница, на которой проводились эксперименты.

Все важные функции отсюда будут в дальнейшем перенесены в папку helpers и использованы в дальнейшем анализе

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/n.barsukov/PycharmProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.models import TOKEN_TYPES
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec

Также внешние библиотеки подключим сразу

In [7]:
import pandas as pd
import json
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

# Код

In [4]:
pathToTokenizeScriptsJson = 'data/parsed-scripts.json'
  
tokenizedSciptsJson = open(pathToTokenizeScriptsJson) 
tokenized_scripts = json.load(tokenizedSciptsJson) 

In [5]:
script = tokenized_scripts['file1082.js']

In [ ]:
sentences = pd.read_csv('News_Final.csv')['Headline'][:10000]
list_of_lists = [[str(word) for word in str(sentence).split()] for sentence in sentences]

Теперь бы нужно настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [ ]:
model = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=5,
    
    # финальный размер словаря
    max_final_vocab=10000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 0, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

In [ ]:
model.wv.most_similar('Obama')